In [14]:
from PIL import Image
import numpy as np
import brotli
import struct
import base64
import io
import heapq
from collections import Counter

def delta_decode(data):
    decoded = [data[0]]
    for i in range(1, len(data)):
        decoded.append(np.uint8((int(decoded[i - 1]) + data[i]) % 256))
    return decoded

def rle_decode(encoded):
    decoded = bytearray()
    i = 0
    while i < len(encoded):
        value = encoded[i]
        count = struct.unpack(">H", encoded[i + 1:i + 3])[0]
        decoded.extend([value] * count)
        i += 3
    return bytes(decoded)

def bitunpack_4bit(packed_data, original_length):
    unpacked = []
    for byte in packed_data:
        high_nibble = (byte >> 4) & 0x0F
        low_nibble = byte & 0x0F
        unpacked.append(high_nibble)
        unpacked.append(low_nibble)
    return unpacked[:original_length]

def huffman_decode(encoded_data, tree):
    decoded_data = []
    current_node = tree
    bit_string = ''.join(format(byte, '08b') for byte in encoded_data)
    for bit in bit_string:
        if bit == '0':
            current_node = current_node.left
        else:
            current_node = current_node.right
        if current_node.char is not None:
            decoded_data.append(current_node.char)
            current_node = tree  # Reset to root
    return bytes(decoded_data)

def build_huffman_tree_from_codebook(codebook):
    root = Node()
    for char, code in codebook.items():
        current_node = root
        for bit in code:
            if bit == '0':
                if current_node.left is None:
                    current_node.left = Node()
                current_node = current_node.left
            else:
                if current_node.right is None:
                    current_node.right = Node()
                current_node = current_node.right
        current_node.char = char
    return root

def lzw_decode(encoded_bytes):
    codes = [struct.unpack('>H', encoded_bytes[i:i+2])[0] for i in range(0, len(encoded_bytes), 2)]
    dictionary = {i: bytes([i]) for i in range(256)}
    w = bytes([codes.pop(0)])
    result = bytearray(w)
    for k in codes:
        if k in dictionary:
            entry = dictionary[k]
        elif k == len(dictionary):
            entry = w + w[:1]
        else:
            raise ValueError('Bad compressed k: %s' % k)
        result += entry
        dictionary[len(dictionary)] = w + entry[:1]
        w = entry
    return bytes(result)

def unpack_bw_1bit(packed_data, shape):
    total_pixels = shape[0] * shape[1]
    flat = []

    for byte in packed_data:
        for i in range(7, -1, -1):
            flat.append((byte >> i) & 1)
            if len(flat) == total_pixels:
                break

    return np.array(flat, dtype=np.uint8).reshape(shape)

def delta_decode_rgb(delta_array):
    delta_array = delta_array.astype(np.int16)
    image_array = np.zeros_like(delta_array, dtype=np.uint8)
    image_array[0] = delta_array[0]
    for i in range(1, len(delta_array)):
        image_array[i] = (image_array[i - 1] + delta_array[i]) % 256
    return image_array

def bitunpack_rgb(packed):
    bits = ''.join(format(byte, '08b') for byte in packed)
    pixels = []

    # Iterate through the bit string, extracting 5 bits at a time for each channel
    i = 0
    while i + 15 <= len(bits):
        r = int(bits[i:i + 5], 2)
        g = int(bits[i + 5:i + 10], 2)
        b = int(bits[i + 10:i + 15], 2)

        # Expand from 5 bits to 8 bits (lossless)
        r = (r << 3) | (r >> 2)
        g = (g << 3) | (g >> 2)
        b = (b << 3) | (b >> 2)

        pixels.append([r, g, b])
        i += 15

    return np.array(pixels, dtype=np.uint8)

def delta_decode_alpha(delta_channel):
    delta_channel = delta_channel.astype(np.int16)
    alpha_channel = np.zeros_like(delta_channel, dtype=np.uint8)
    alpha_channel[0] = delta_channel[0]
    for i in range(1, len(delta_channel)):
        alpha_channel[i] = (alpha_channel[i - 1] + delta_channel[i]) % 256
    return alpha_channel

class Node:
    def __init__(self, char=None, freq=None):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    def __lt__(self, other):
        return self.freq < other.freq

def decode_lix_image(lix_path, output_path=None):
    with open(lix_path, "rb") as f:
        data = f.read()

    magic = data[:4]
    width, height, img_type, method = struct.unpack(">HHBB", data[4:10])
    compressed_data = data[10:]

    if magic == b"LIXG":
        # Grayscale
        if method == 0:
            decoded = delta_decode(compressed_data)
        elif method == 1:
            rle_decoded = rle_decode(compressed_data)
            decoded = delta_decode(rle_decoded)
        elif method == 2:
            # Huffman
            raise NotImplementedError("Huffman decoding for grayscale not yet fully implemented")
        elif method == 3:
            delta_decoded = lzw_decode(compressed_data)
            decoded = delta_decode(delta_decoded)
        else:
            raise ValueError(f"Unknown grayscale method: {method}")
        img_array = np.array(decoded, dtype=np.uint8).reshape((height, width))
        img = Image.fromarray(img_array, mode="L")

    elif magic == b"LIXB":
        # Black & White
        print("Decoding LIXB (Black & White)")
        print(f"  Width: {width}, Height: {height}")
        print(f"  Compressed Data Length: {len(compressed_data)}")

        rle_decoded = rle_decode(compressed_data)
        print(f"  RLE Decoded Length: {len(rle_decoded)}")

        unpacked = unpack_bw_1bit(rle_decoded, (height, width))
        print(f"  Unpacked Length: {len(unpacked)}")

        expected_pixels = width * height
        print(f"  Expected Pixels: {expected_pixels}")

        img_array = unpacked.reshape((height, width)) * 255  # Scale back to 0/255
        img = Image.fromarray(img_array.astype(np.uint8), mode="L")  # Create grayscale for B/W
        print("LIXB decoded successfully")

    elif magic == b"LIXF":
        # RGB
        if img_type != 0x03:
            raise ValueError("Expected image type 0x03 (RGB)")

        if method == 1:
            # Delta + Huffman
            flat_bytes = huffman_decode(compressed_data, build_huffman_tree(bytearray(compressed_data)))
            delta_array = np.frombuffer(flat_bytes, dtype=np.uint8).reshape(-1, 3)
            img_array = delta_decode_rgb(delta_array)
        elif method == 2:
            # Bitpack + Brotli
            uncompressed = brotli.decompress(compressed_data)
            img_array = bitunpack_rgb(uncompressed)
        elif method == 3:
            # Delta + Bitpack + Brotli
            uncompressed = brotli.decompress(compressed_data)
            bitunpacked = bitunpack_rgb(uncompressed)
            img_array = delta_decode_rgb(bitunpacked)
        else:
            raise ValueError(f"Unknown RGB method: {method}")

        img_array = img_array.reshape((height, width, 3))
        img = Image.fromarray(img_array, mode="RGB")

    elif magic == b"LIXA":
        # RGBA
        rgb_compressed = data[10: - (len(compressed_data) if method else 0)]
        alpha_compressed = data[-len(compressed_data):] if method else b''

        #RGB DECODING
        rgb_uncompressed = brotli.decompress(rgb_compressed)
        rgb_pixels = bitunpack_rgb(rgb_uncompressed)
        rgb_array = rgb_pixels.reshape((height, width, 3))
        img_rgb = Image.fromarray(rgb_array.astype(np.uint8), mode="RGB")
        img = img_rgb.convert("RGBA") #Start with RGB and add Alpha

        #ALPHA DECODING
        if method == 1:
            alpha_decompressed = brotli.decompress(alpha_compressed)
            alpha_delta_decoded = delta_decode_alpha(np.frombuffer(alpha_decompressed, dtype=np.uint8))
            alpha_channel = alpha_delta_decoded.reshape((height, width))
            img.putalpha(Image.fromarray(alpha_channel.astype(np.uint8), mode='L'))

    else:
        raise ValueError("Invalid LIX file format.")

    if output_path:
        img.save(output_path)
        print(f"✅ Decoded image saved to {output_path}")
    return img

# Example usage
decode_lix_image("gray.lix", "decoded_gray.png")
decode_lix_image("bw.lix", "decoded_bw.png")
decode_lix_image("rgb.lix", "decoded_rgb.png")
decode_lix_image("rgba.lix", "decoded_rgba.png")

✅ Decoded image saved to decoded_gray.png


ValueError: cannot reshape array of size 1099 into shape (128,128)